### Notebook configuration

In [1]:
run_number          = 5548
input_dst_filenames = f"$IC_DATA/runiii/dsts/{run_number}/run_{run_number}_*_dst.h5"
output_dst_filename = f"$IC_DATA/Kr/dst_{run_number}.root.h5"

### Imports

In [2]:
import os
import glob

import numpy  as np
import tables as tb

import invisible_cities.io.dst_io as dstio

### Initialization

In [3]:
input_dst_filenames = os.path.expandvars(input_dst_filenames)
output_dst_filename = os.path.expandvars(output_dst_filename)

In [4]:
column_map = dict(
evt = "event",
tsp = "time",

ns1 = "nS1",
es1 = "S1e",
hs1 = "S1h",
ws1 = "S1w",
ts1 = "S1t",

ns2 = "nS2",
es2 = "S2e",
hs2 = "S2h",
ws2 = "S2w",
ts2 = "S2t",
qs2 = "S2q",
xs2 = "X",
ys2 = "Y",
nsi = "Nsipm",
)

### Load dsts

In [5]:
dst = dstio.load_dsts(glob.glob(input_dst_filenames), "DST", "Events")

### Rename columns

In [6]:
dst = dst.rename(columns=column_map)

### Delete index column

In [7]:
del dst["index"]

### Add missing columns

In [8]:
dst["S2h"]  = 0

dst["DT"]   = dst.S2t - dst.S1t
dst["Z"]    = dst.DT
dst["R"]    = (dst.X**2 + dst.Y**2)**0.5
dst["Phi"]  = np.arctan2(dst.Y, dst.X)

dst["Xrms"] = 0
dst["Yrms"] = 0

### Filter out multiple-S1 events

In [9]:
one_S1 = dst.nS1 == 1

number_of_evts  = np.unique(dst.event.values).size
number_of_oneS1 = np.count_nonzero(one_S1)
percentage      = 100 * number_of_oneS1 / number_of_evts

print(f"Total number of events: {number_of_evts} ")
print(f"Total number of events with 1 S1: {number_of_oneS1} = {percentage:.1f} %")

Total number of events: 144015 
Total number of events with 1 S1: 46732 = 32.4 %


### Create new dst

In [10]:
# Unfortunately, this method can't set a specific name to the table or its title.
# It also includes an extra column ("index") which I can't manage to remove.
subdst = dst[one_S1]
subdst.to_hdf(output_dst_filename,
              key     = "DST"  , mode         = "w",
              format  = "table", data_columns = True,
              complib = "zlib" , complevel    = 4)

# Workaround to re-establish the name of the table and its title
with tb.open_file(output_dst_filename, "r+") as f:
    f.rename_node(f.root.DST.table, "Events")
    f.root.DST.Events.title = "Events"